In [10]:
import sys
sys.path.append("./miniaturautonomie_lanedetection/include/")
sys.path.append("./include/")
from tensorflow.keras.models import load_model, Model
import tensorflow as tf
import os
import fnmatch
import numpy as np
import random
import render
import cv2
import albumentations as Alb
from DataGenerator import DataGenerator
import matplotlib.pyplot as plt
import coremltools as ct

In [4]:
output_dir = f"./output/tiny_ft_g125_a7"
model_name = "model"
model = load_model(f"{output_dir}/{model_name}.h5", custom_objects={'iou_score':None, 
                                                            'f1_score':None, 
                                                            'precision_m':None, 
                                                            'recall_m':None,
                                                            'tversky_loss': None,
                                                            'focal_tversky_loss': None,
                                                            'weighted_ce': None,
                                                            'dice_loss': None})
input_img_size = (480, 640)
output_width = 640 
output_height = 224 
input_width = 640 
input_height = 224
number_classes = 7 # outer, middle_curb, guide_lane, solid_lane, hold_line, zebra, background
packages = ['knuff_main1', 'knuff_main2', 'knuff_main3', 'knuff_hill', 'highway', 'knuff_main5', 'knuff_main6']
image_path = './data/'
annotation_path = './annotation_v3.0/'

# generate absolute list of all img and masks paths
image_paths = []
mask_paths = []
# for package
for index in range(len(packages)):
    image_base_path = f"{image_path}{packages[index]}/"
    masks_base_path = f"{annotation_path}{packages[index]}/masks/"

    file_list = os.listdir(masks_base_path)
    pattern = '*.png'
    for filename in file_list:
        if fnmatch.fnmatch(filename, pattern):
            mask_paths.append(os.path.join(masks_base_path, filename))
            image_name = filename[:len(filename)-3]+"jpg"
            image_paths.append(os.path.join(image_base_path, image_name))
            
size_all = len(image_paths)
random.Random(size_all).shuffle(image_paths)
random.Random(size_all).shuffle(mask_paths)

random.Random(size_all).shuffle(image_paths)
random.Random(size_all).shuffle(mask_paths)

random.Random(size_all).shuffle(image_paths)
random.Random(size_all).shuffle(mask_paths)

2022-05-10 13:25:56.809481: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 13:25:56.815511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 13:25:56.816056: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 13:25:56.816927: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Change resolution

In [10]:
# change image resolution if needed
import architectures.unet as unet
import architectures.small_unet as small_unet

new_model = unet.vgg("res_conversion", 96, 224, number_classes, None)
new_model.set_weights(model.get_weights())
model.save(f"{output_dir}/model_224_96.h5")

ModuleNotFoundError: No module named 'architectures'

# TFLite with 8Bit quantization

In [5]:
number_of_rep = 100
params = {
    'batch_size': number_of_rep,
    'input_img_size': input_img_size,
    'target_img_size': (output_height, output_width),
    'shuffle': True,
    'n_channels': number_classes,
    'augmentation': False
}
rep_gen = DataGenerator(image_paths, mask_paths, **params)
representative_imgs = image_paths[:number_of_rep]
representative_masks = mask_paths[:number_of_rep]

imgs, masks = rep_gen.data_generation(representative_imgs, representative_masks)

In [8]:
# zero_img = np.zeros((1,output_height, output_width,3), dtype=np.float32)
# one_img = np.ones((1,output_height, output_width,3), dtype=np.float32)

# imgs = np.concatenate((imgs, zero_img))
# imgs = np.concatenate((imgs, one_img))

def representative_dataset():
    for img in imgs:
        #img = cv2.resize(img, (224, 96))
        data = np.expand_dims(img, axis=0)
        yield [data.astype(np.float32)]
        
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8  # or tf.uint8
converter.inference_output_type = tf.uint8  # or tf.uint8
converter.experimental_new_quantizer = True
# converter.default_ranges_stats = (0,255)
tflite_quant_model = converter.convert()
open(f"{output_dir}/{model_name}.tflite", "wb").write(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmpy8n6lsna/assets


INFO:tensorflow:Assets written to: /tmp/tmpy8n6lsna/assets
/home/danielriege/.local/share/virtualenvs/lane_detection-lywnBSQa/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2022-05-10 13:28:12.333455: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-05-10 13:28:12.333480: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-05-10 13:28:12.333663: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpy8n6lsna
2022-05-10 13:28:12.360597: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-05-10 13:28:12.360629: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpy8n6lsna
2022-05-10 13:28:12

TypeError: EndVector() takes 1 positional argument but 2 were given

In [ ]:
# Initialize the interpreter
interpreter = tf.lite.Interpreter(model_path=f"{output_dir}/model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

imgs = imgs[:2]
masks = masks[:2]

# Check if the input type is quantized, then rescale input data to uint8
outputs = np.zeros((imgs.shape[0], imgs.shape[1], imgs.shape[2], 7), dtype=np.uint8)
lite_imgs = []
for i, img in enumerate(imgs):
    
    input_scale, input_zero_point = input_details["quantization"]
    print(f"{input_scale} : {input_zero_point}")
    lite_img = np.array(img / input_scale + input_zero_point, dtype=np.uint8)
    lite_imgs.append(lite_img)
    lite_img = np.expand_dims(lite_img, axis=0)
    interpreter.set_tensor(input_details["index"], lite_img)
    interpreter.invoke()
    outputs[i] = interpreter.get_tensor(output_details["index"])
        

predictions = model.predict(imgs)
thres_value = 0.6

f, axs = plt.subplots(len(predictions), 4, figsize=(20,4))
for i, prediction in enumerate(predictions):
    # show input img
    axs[i,0].imshow(imgs[i])
    axs[i,0].title.set_text('Input')
    
    # show prediction
    ## clip to 0 or 1 with thres
    clipped_pred = np.where(prediction > thres_value, 1, 0)
    rgb_pred = render.render_rgb(clipped_pred)
    
    axs[i,1].imshow(rgb_pred)
    axs[i,1].title.set_text('float32')
    
    # show input img
    axs[i,2].imshow(lite_imgs[i])
    axs[i,2].title.set_text('Input uint8 quantized')
    
    ## clip to 0 or 1 with thres
    clipped_pred = np.where(outputs[i] > thres_value, 1, 0)
    rgb_pred = render.render_rgb(clipped_pred)
    
    axs[i,3].imshow(rgb_pred)
    axs[i,3].title.set_text('uint8')
    
f.savefig('tflite_0.027.png')

# MLModel

In [ ]:
mlmodel = ct.convert(model)
mlmodel.save(f"{output_dir}/{model_name}.mlmodel")

In [ ]:
# verify
mlmodel = ct.models.MLModel(f"{output_dir}/{model_name}.mlmodel")

imgs = imgs[:2]
masks = masks[:2]

# Check if the input type is quantized, then rescale input data to uint8
outputs = np.zeros((imgs.shape[0], imgs.shape[1], imgs.shape[2], 7), dtype=np.uint8)
lite_imgs = []
for i, img in enumerate(imgs):
    
    input_scale, input_zero_point = input_details["quantization"]
    print(f"{input_scale} : {input_zero_point}")
    lite_img = np.array(img / input_scale + input_zero_point, dtype=np.uint8)
    lite_imgs.append(lite_img)
    lite_img = np.expand_dims(lite_img, axis=0)
    interpreter.set_tensor(input_details["index"], lite_img)
    interpreter.invoke()
    outputs[i] = interpreter.get_tensor(output_details["index"])
        

predictions = model.predict(imgs)
thres_value = 0.6

f, axs = plt.subplots(len(predictions), 4, figsize=(20,4))
for i, prediction in enumerate(predictions):
    # show input img
    axs[i,0].imshow(imgs[i])
    axs[i,0].title.set_text('Input')
    
    # show prediction
    ## clip to 0 or 1 with thres
    clipped_pred = np.where(prediction > thres_value, 1, 0)
    rgb_pred = render.render_rgb(clipped_pred)
    
    axs[i,1].imshow(rgb_pred)
    axs[i,1].title.set_text('float32')
    
    # show input img
    axs[i,2].imshow(lite_imgs[i])
    axs[i,2].title.set_text('Input uint8 quantized')
    
    ## clip to 0 or 1 with thres
    clipped_pred = np.where(outputs[i] > thres_value, 1, 0)
    rgb_pred = render.render_rgb(clipped_pred)
    
    axs[i,3].imshow(rgb_pred)
    axs[i,3].title.set_text('uint8')
    
f.savefig('tflite_0.027.png')